## Loading/Preprocessing the NHS TRUD DATA into a SQLite database

In [56]:
#Loading the TRUD Data into a SQLite database
#Loading and Imporrting relavent packages
import sqlite3
import pandas as pd

# Processing and Loading the description table
decrip_df = pd.read_csv("/Users/laurakhaukha/Desktop/Diss_data/SnomedCT_UKClinicalRF2_PRODUCTION_20241120T000001Z/Snapshot/Terminology/sct2_Description_UKCLSnapshot-en_GB1000000_20241120.txt", sep="\t", dtype=str)
decrip_df.columns = map(str.lower, decrip_df.columns) #standardise column names, making everything lowercase 
decrip_df = decrip_df[decrip_df["active"] == "1"] #Filtering the active entries to include for the model, take away the inactive ones
#/Users/laurakhaukha/Desktop/Diss_data/SnomedCT_UKClinicalRefsetsRF2_PRODUCTION_20241120T000001Z/Snapshot/Terminology/sct2_Description_UKCRSnapshot-en_GB1000000_20241120.txt

# Processing and loading the relationships table
rela_df = pd.read_csv("/Users/laurakhaukha/Desktop/Diss_data/SnomedCT_UKClinicalRF2_PRODUCTION_20241120T000001Z/Snapshot/Terminology/sct2_Relationship_UKCLSnapshot_GB1000000_20241120.txt", sep="\t", dtype=str)
rela_df.columns = map(str.lower, rela_df.columns) #standardise column names, making everything lowercase 
rela_df = rela_df[rela_df["active"] == "1"]  #Filtering the active entries to include for the model, take away the inactive ones

# # Preprocessing and loading the language tables
# langu_df = pd.read_csv("/Users/laurakhaukha/Desktop/Diss_data/SnomedCT_UKClinicalRF2_PRODUCTION_20241120T000001Z/Snapshot/Refset/Language/der2_cRefset_LanguageUKCRSnapshot-en_GB1000000_20241120.txt",sep="\t", dtype=str)
# langu_df.columns = langu_df.columns.str.lower()
# langu_df = langu_df[langu_df["active"] == "1"]

# Creating the SQLite database
conn = sqlite3.connect("snomedct_data.db")

In [52]:
#Writing the description and relationship tables to the database
decrip_df.to_sql("TRUD_descriptiontable", conn, if_exists="replace", index=False)
rela_df.to_sql("TRUD_relationshiptable", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

# Verifying the database 
conn = sqlite3.connect("snomedct_data.db")
cur = conn.cursor() #Establisshing the Cursor object, used to execute SQL commands

#Listing the tables 
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())

#Looking at the contents
sqllite_df = pd.read_sql_query("SELECT * FROM TRUD_descriptiontable LIMIT 10", conn)
print(sqllite_df.head())
conn.close() #Closing the connecting

#I do not need moduleid and casesignificanceid for the prompts 
# sourceid and concpetid Are the same 
# relationship table descirebs the child and parent relationships 

[('TRUD_descriptiontable',), ('TRUD_relationshiptable',)]
            id effectivetime active            moduleid       conceptid  \
0  11000000117      20040131      1  999000011000000103  20881000000105   
1  21000000111      20040131      1  999000011000000103     11000000101   
2  21000237110      20230607      1  999000011000000103    861000237103   
3  31000000113      20040131      1  999000011000000103  20891000000107   
4  31000237112      20230607      1  999000011000000103   6561000237105   

  languagecode              typeid  \
0           en  900000000000013009   
1           en  900000000000013009   
2           en  900000000000003001   
3           en  900000000000013009   
4           en  900000000000013009   

                                                term  casesignificanceid  
0                           Read Code Administration  900000000000020002  
1                         Read Code Administration 1  900000000000020002  
2  Mass concentration of drug in urin

## EDA (Exploriratory Data Analysis) 

In [62]:
conn_1 = sqlite3.connect("snomedct_data.db")
pd.read_sql_query("SELECT * FROM TRUD_descriptiontable LIMIT 5", conn_1)

,id,effectivetime,active,moduleid,conceptid,languagecode,typeid,term,casesignificanceid
0,11000000117,20040131,1,999000011000000103,20881000000105,en,900000000000013009,Read Code Administration,900000000000020002
1,21000000111,20040131,1,999000011000000103,11000000101,en,900000000000013009,Read Code Administration 1,900000000000020002
2,21000237110,20230607,1,999000011000000103,861000237103,en,900000000000003001,Mass concentration of drug in urine (observabl...,900000000000020002
3,31000000113,20040131,1,999000011000000103,20891000000107,en,900000000000013009,Read Code Administration 2,900000000000020002
4,31000237112,20230607,1,999000011000000103,6561000237105,en,900000000000013009,Calcium percent in calculus,900000000000020002


In [64]:
#DTA, finding the concept id that is the most common: 
concptcounts = decrip_df["conceptid"].value_counts().reset_index()
print(F"{concptcounts}: Concpet ID counts")
concptcounts.head(21)

                 conceptid  count
0          466891000000108     20
1          386721000000105     14
2          256691000000101     14
3          413681000000103     14
4          466871000000109     13
...                    ...    ...
100386     431781000000100      2
100387     390911000000104      2
100388     414021000000105      2
100389  999003041000000106      2
100390    1110451000000102      1

[100391 rows x 2 columns]: Concpet ID counts


,conceptid,count
0,466891000000108,20
1,386721000000105,14
2,256691000000101,14
3,413681000000103,14
4,466871000000109,13
5,470791000000108,12
6,368851000000106,12
7,401991000000109,12
8,468611000000102,12
9,431821000000108,12


In [49]:
#DTA
concept_id__ = "466891000000108"
query = f"""SELECT term, typeid FROM TRUD_descriptiontable WHERE conceptid = '{concept_id__}'"""
pd.read_sql_query(query, conn)

,term,typeid
0,([X]Deoxybarbiturates causing adverse effects ...,900000000000003001
1,[X] Adverse reaction to tetracaine,900000000000013009
2,[X] Adverse reaction peripheral nerve/plexus-b...,900000000000013009
3,([X]Deoxybarbiturates causing adverse effects ...,900000000000013009
4,([X]Deoxybarbiturates causing adverse effects ...,900000000000013009
5,[X] Adverse reaction to surface and infiltrati...,900000000000013009
6,[X] Adverse reaction to lignocaine hydrochloride,900000000000013009
7,[X] Adverse reaction to local anaesthetics NOS,900000000000013009
8,[X] Adverse reaction to local anesthetics NOS,900000000000013009
9,[X] Adverse reaction to other local anaesthetics,900000000000013009


In [ ]:
#Preporcoessing the data 

# map type ids to the type label: 900000000000003001(FSN), 900000000000013009: (Synonym)
# Potentiallly Join relashionship and description table on concept id and source id 
#Filter to the UK- preferred terms using the language table : Filter the synonoym with typeid 
#Keep useful columns for the prompts: conceptid, term, typeid, sourceid, destinationid, typeid 
#Creating Prompts 
# Example 
# #Defining the tables used in the prompts
# schem = """ Given the tables:
#      TRUD_descriptiontable(conceptid, term, typeid) TRUD_relationshiptable(sourceid, destinationid, typeid)
#      Provide a SQL Query to answer the following question """

# # Create an expected SQL query
# # Find concept ids that are rich in info 
# Given tables : ... 
# What are the synonyms for "term" 
# What is the code for term " "
# Count the number of synonyms, concepts etc...
# Get all the descrioptions based on the type id 

## Creating the PROMPTS

In [38]:
# Category 1: Looking up concepts 
# 10 SQL-NL prompts 
slq_promts_category1 = [
    ("What is the preferred term for concept ID 466891000000108?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108' AND typeid = '900000000000013009'"),
    ("What is the full name for concept ID 466891000000108?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108' AND typeid = '900000000000003001'"),
    ("Find the preferred name of concept ID 386721000000105.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '386721000000105' AND typeid = '900000000000013009'"),
    ("What is the English name of concept ID 386721000000105 ?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '386721000000105' AND languagecode = 'en' AND typeid = '900000000000003001'"),
    ("Give the preferred description of concept ID 386721000000105.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '386721000000105' AND typeid = '900000000000003001'"),
    ("What description is used as the preferred term for concept ID 466871000000109?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466871000000109' AND typeid = '900000000000013009'"),
    ("What is the label of concept 466871000000109?",
     "SELECT term FROM TRUD_descriptiontable WHERE concecptid = '466871000000109' AND typeid = '900000000000013009'"),
    ("Get the preferred English label for concept ID 413681000000103	.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '413681000000103	' AND typeid = '900000000000013009' AND languagecode = 'en'"),
    ("What name is preferred for the concept 413681000000103?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '413681000000103' AND typeid = '900000000000013009'"),
    ("Give me the preferred description for concept 3413681000000103.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '3413681000000103' AND typeid = '900000000000003001'")]
# Add another 20 to this one 

In [22]:
# Category 2: Finding Terms and Synonmonymms 
sql_promts_category2 = [
    ("List all synonyms for concept ID 466891000000108.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108'"),
    ("What are the different descriptions for concept 466891000000108?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108'"),
    ("Get all English terms for the concept ID 386721000000105.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '386721000000105' AND languagecode = 'en'"),
    ("Retrieve all labels for concept ID 386721000000105.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '386721000000105'"),
    ("Show all descriptions including synonyms for concept 413681000000103.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '413681000000103'"),    
    ("What are the known terms for concept ID 466891000000108?",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108'"),
    ("Give me all descriptions linked to concept 3413681000000103.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '3413681000000103"),
    ("Get every English label for concept ID 19551000000101.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '19551000000101' AND languagecode = 'en'"),
    ("Return all terms used to describe concept 19551000000101.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '19551000000101'"),
    ("List every description for concept 256691000000101.",
     "SELECT term FROM TRUD_descriptiontable WHERE conceptid = '256691000000101'")]
# Add 10 more for finding synnoyms
#NEED TO involve type id for synonsm 

In [58]:
# Looking at the most popular terms
sqlquery = """SELECT term, COUNT(*) as frequency FROM TRUD_descriptiontable GROUP BY term ORDER BY frequency DESC LIMIT 20"""
temrs_most = pd.read_sql_query(sqlquery, conn)
temrs_most

,term,frequency
0,Hypodermic needle injury,23
1,Accident caused by sharp-edged object,22
2,High cost drugs,16
3,Maltreatment,15
4,High cost chemotherapy drugs,15
5,Transarterial approach,14
6,Reconstruction of cranial defect,14
7,Upper limb,12
8,Excision of lesion of brain tissue,12
9,Distraction osteogenesis of bones of skull,12


In [ ]:
# Category 3: Finding the typeid from the of a term or concept
sql_promts_category3 = [("What type of description is 'Hypodermic needle injury'?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Hypodermic needle injury'"),
    ("Is 'Accident caused by sharp-edged object	' a synonym or a preferred term?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Accident caused by sharp-edged object	'")
    ("Find the typeid for the term 'High cost drugs'.",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'High cost drugs'"),
    ("What is the type for the description 'Maltreatment'?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Maltreatment'"),
    ("Get the description type of 'High cost chemotherapy drugs'.",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'High cost chemotherapy drugs'"),
    ("What is the typeid for the term 'Excision of lesion of brain tissue'?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Excision of lesion of brain tissue'"),
    ("Which type ID corresponds to the term 'Heart rate measurement'?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Heart rate measurement'"),
    ("Is 'Distraction osteogenesis of bones of skull?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Distraction osteogenesis of bones of skull'"),
    ("Find the typeid for 'Operations on bones of skull'.",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Operations on bones of skull"),
    ("What is the description type for 'Operations for disorders of sex development'?",
     "SELECT typeid FROM TRUD_descriptiontable WHERE term = 'Operations for disorders of sex development'")]

In [ ]:
# Category 4: Finding relashionships betweeen concepts 

# sql_prompts_category4 = [
#     ("What are the destination concepts related to concept ID 466891000000108?",
#      "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '466891000000108'"),
#     ("Which concepts are related to 466891000000108?",
#      "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '466891000000108'"),
#     ("Find all source concepts linked to destination ID 466891000000108.",
#      "SELECT sourceid FROM TRUD_relationshiptable WHERE destinationid = '466891000000108'"),
#     ("Get all destination concepts where the relationship type is 'Is a' and source is 466891000000108.")]

In [ ]:
# Here include the caclulation category:  
# Edit
# sql_prompts_category5= ("What are the destination concepts related to concept ID 466891000000108?",
#      "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '466891000000108'"),
#     ("Which concepts are related to 466891000000108?",
#      "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '466891000000108'"),
#     ("Find all source concepts linked to destination ID 466891000000108.",
#      "SELECT sourceid FROM TRUD_relationshiptable WHERE destinationid = '466891000000108'"),
#     ("Get all destination concepts where the relationship type is 'Is a' and source is 466891000000108.",
#      "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '223366009' AND typeid = '466891000000108'"),
#     #  "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '861000237103' AND typeid = '116680003'"),
#     # ("List source concepts with a 'Causative agent' relationship to 410942007.",
#     #  "SELECT sourceid FROM TRUD_relationshiptable WHERE destinationid = '410942007' AND typeid = '246075003'"),
#     # ("Return destination IDs related to 11000000101 through any type of relationship.",
#     #  "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '11000000101'"),
#     # ("What are the concept IDs that 31000000113 is related to?",
#     #  "SELECT destinationid FROM TRUD_relationshiptable WHERE sourceid = '31000000113'")]

## Evalutating the prompts for the T5 model 

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.4 MB/s eta 0:00:00a 0:00:01


In [24]:
!pip install sentencepiece

In [35]:
# Seting the Model To English
# transformers-cli env
# transformers-cli cache --force-clear

In [1]:
# Importing the relevaent libaries
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Loading the model and the relevant tokeniser 
t5_tokeniser = T5Tokenizer.from_pretrained("t5-small")
T5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [60]:
# Looping through each prompt tokenisesing them and feeding them into the model to generate output
# FIX THISL: MODELS OUPUT IN GERMNAN:
for i, (propt, esql) in enumerate(slq_promts_category1 ):
    input = t5_tokeniser(propt, return_tensors="pt").input_ids
    outputs_ = T5_model.generate(input , max_length=100)
    ouput_sql = t5_tokeniser.decode(outputs_[0], skip_special_tokens=True)
    print(f"Example {i+1}")
    print("Provided Prompt: ", propt)
    print("Expected SQL: ", esql)
    print("Output SQL: ", ouput_sql)

Example 1
Provided Prompt:  What is the preferred term for concept ID 466891000000108?
Expected SQL:  SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108' AND typeid = '900000000000003001'
Output SQL:  Was ist der Begriff für den Begriff ID 466891000000108?
Example 2
Provided Prompt:  What is the full name for concept ID 466891000000108?
Expected SQL:  SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108' AND typeid = '900000000000003001'
Output SQL:  Was ist der Name voll für den Begriff ID 466891000000108?
Example 3
Provided Prompt:  Find the preferred name of concept ID 466891000000108.
Expected SQL:  SELECT term FROM TRUD_descriptiontable WHERE conceptid = '466891000000108' AND typeid = '900000000000013009'
Output SQL:  Find the preferred name of concept ID 466891000000108.
Example 4
Provided Prompt:  What is the English name of concept ID 466891000000108 ?
Expected SQL:  SELECT term FROM TRUD_descriptiontable WHERE conceptid = '3867210000